In [ ]:
# %% [markdown]
# # Lab 8: Threads & Conversation Management
#
# **Goal:** Understand how to manage multiple, independent runs of a graph concurrently using "threads". This is essential for any real-world application where you need to handle multiple users or tasks at the same time.
#
# ---

# %% [markdown]
# ## Setup
#
# Let's create a simple, linear graph to use for our demonstration. This is the same graph we built in Lab 5.

# %%
# Install required packages
!pip install langgraph --quiet
print("Libraries installed.")

# Basic imports
from langgraph.graph import StateGraph, END
from typing import TypedDict, List

# Define the state and nodes
class ResumeState(TypedDict):
    resume_text: str
    decision: str

def process_resume(state: ResumeState) -> dict:
    """A simple node that makes a decision based on resume content."""
    print(f"---Processing Resume: '{state['resume_text']}'---")
    if "senior" in state["resume_text"].lower():
        decision = "Interview"
    else:
        decision = "Reject"
    print(f"Decision: {decision}")
    return {"decision": decision}

# Build and compile the graph
workflow = StateGraph(ResumeState)
workflow.add_node("process", process_resume)
workflow.set_entry_point("process")
workflow.add_edge("process", END)
app = workflow.compile()
print("\nSimple graph compiled and ready.")

# %% [markdown]
# ---
# ## Part A: Understanding Threads
#
# In LangGraph, a "thread" is simply an independent sequence of operations within the same graph. Each thread has its own unique ID and its own separate state history.
#
# Think of it like a multi-lane highway:
# -   The **Graph** is the highway itself.
# -   Each **Thread** is a car driving on that highway.
# -   The **State** is the car's current position and status.
#
# Each car (thread) can be in a different lane, at a different speed, and at a different point on the highway, but they are all following the same road rules (the graph's structure).
#
# You manage threads by passing a `config` dictionary with a `thread_id` to the `invoke` or `stream` method.

# %% [markdown]
# ---
# ## Part B: A Single Thread Example
#
# Let's process one candidate's resume. We'll assign this process a unique `thread_id`. LangGraph will now store the history of this specific run under that ID.

# %%
# Define the input for our first candidate
candidate_john = {
    "resume_text": "John Doe is a senior Python developer."
}

# Define the configuration for this thread
config_john = {"thread_id": "candidate_john_doe_123"}

# Invoke the graph with the input and the thread-specific config
result_john = app.invoke(candidate_john, config=config_john)

print(f"\nResult for {config_john['thread_id']}: {result_john['decision']}")

# %% [markdown]
# Behind the scenes, LangGraph has now stored the state transitions for the thread `"candidate_john_doe_123"`. If we were to invoke the graph again with the same `thread_id`, it would be a continuation of this run (useful for conversational agents).
#
# ---

# %% [markdown]
# ## Part C: Multiple Concurrent Threads
#
# Now, let's process a list of candidates. We will loop through them and invoke the *same* compiled graph (`app`) for each one, but we will give each invocation a *different* `thread_id`. This ensures that each candidate's process is handled independently and their states do not interfere with each other.

# %%
# A list of candidates to process
candidates = [
    {
        "id": "candidate_jane_456",
        "resume": "Jane Smith is a senior Java engineer."
    },
    {
        "id": "candidate_bob_789",
        "resume": "Bob Wilson is a junior developer."
    },
    {
        "id": "candidate_alice_012",
        "resume": "Alice Williams is a senior project manager."
    }
]

final_results = []

print("---Processing Multiple Candidates Concurrently---")

# Process each candidate in their own independent thread
for candidate in candidates:
    # Prepare the input state for this candidate
    input_state = {"resume_text": candidate["resume"]}

    # Create a unique config for this candidate's thread
    config = {"thread_id": candidate["id"]}

    # Invoke the graph
    result = app.invoke(input_state, config=config)

    # Store the final decision
    final_results.append({
        "id": candidate["id"],
        "decision": result["decision"]
    })

print("\n---All Candidates Processed---")
for res in final_results:
    print(f"Thread ID: {res['id']}, Final Decision: {res['decision']}")


# %% [markdown]
# ### Key Benefits of Using Threads:
#
# * **Isolation:** The state of one candidate's application will never affect another's.
# * **Concurrency:** You can process many candidates at the same time (especially with `astream`).
# * **Stateful Conversations:** For a single user, you can use the same `thread_id` over multiple interactions to maintain memory and context, allowing you to pause and resume complex workflows.
# * **Debugging:** It is much easier to trace the history of a specific run when it has a unique ID.